In [2]:
# Colab Notebook: VAE + Zero-Shot Pipeline with Train/Test Split & Final Validation


# Cell 1: Install dependencies
!pip install --quiet \
    tensorflow \
    sentence-transformers \
    transformers \
    torch \
    scikit-learn \
    datasets \
    faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 14.3 MB/s eta 0:00:00


In [3]:
# Cell 2: Imports & Config
import numpy as np
import pandas as pd
import torch
import faiss

import tensorflow as tf
import tensorflow.keras.backend as K

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report
)

from sentence_transformers import SentenceTransformer
from transformers import pipeline
from datasets import Dataset


In [4]:
# Cell 3: VAE Layers & Helper
@tf.keras.utils.register_keras_serializable()
def sampling(args):
    mean, log_var = args
    eps = K.random_normal(tf.shape(mean))
    return mean + tf.exp(0.5 * log_var) * eps

@tf.keras.utils.register_keras_serializable(package="Custom")
class VAELossLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        orig, recon, mean, log_var = inputs
        recon_loss = tf.reduce_sum(tf.square(orig - recon), axis=1)
        kl_loss    = -0.5 * tf.reduce_sum(1 + log_var - tf.square(mean) - tf.exp(log_var), axis=1)
        self.add_loss(tf.reduce_mean(recon_loss + kl_loss))
        return recon

def build_vae(input_dim):
    inp = tf.keras.Input((input_dim,))
    x = tf.keras.layers.Dense(64, activation="relu")(inp)
    x = tf.keras.layers.Dense(32, activation="relu")(x)
    z_mean   = tf.keras.layers.Dense(4)(x)
    z_logvar = tf.keras.layers.Dense(4)(x)
    z        = tf.keras.layers.Lambda(sampling)([z_mean, z_logvar])

    latent = tf.keras.Input((4,))
    y = tf.keras.layers.Dense(32, activation="relu")(latent)
    y = tf.keras.layers.Dense(64, activation="relu")(y)
    out = tf.keras.layers.Dense(input_dim)(y)
    decoder = tf.keras.Model(latent, out)

    recon = decoder(z)
    loss_out = VAELossLayer()([inp, recon, z_mean, z_logvar])
    vae = tf.keras.Model(inp, loss_out)
    vae.compile(optimizer='adam')
    return vae

def load_vae(path, dim):
    try:
        m = tf.keras.models.load_model(path, custom_objects={"sampling":sampling, "VAELossLayer":VAELossLayer})
        if m.input_shape[1]!=dim:
            return None
        return m
    except:
        return None


In [5]:
# Cell 4: Load & Split Data
df = pd.read_csv("linux_memory_binarized_30.csv", low_memory=False)
features = ['ts','PID','MINFLT','MAJFLT','VSTEXT','VSIZE','RSIZE','VGROW','RGROW','MEM']
df = df.dropna(subset=features+['type']).reset_index(drop=True)

# Train/test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['type'], random_state=0)

# Extract X, y
scaler = StandardScaler()
X_train = scaler.fit_transform(train_df[features].astype(float)).astype(np.float32)
y_train = train_df['type'].astype(int).values

X_test  = scaler.transform(test_df[features].astype(float)).astype(np.float32)
y_test  = test_df['type'].astype(int).values

print("Train samples:", X_train.shape[0], "Test samples:", X_test.shape[0])


Train samples: 240000 Test samples: 60000


In [6]:
# Cell 5: Train/Load VAE on Train Normals
normal_mask = (y_train==0)
vae = load_vae("vae_model.keras", X_train.shape[1])
if vae is None:
    vae = build_vae(X_train.shape[1])
    vae.fit(
        X_train[normal_mask], X_train[normal_mask],
        epochs=20, batch_size=32, validation_split=0.1,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(patience=2)
        ], verbose=1
    )
    vae.save("vae_model.keras")


Epoch 1/20
5929/5929 ━━━━━━━━━━━━━━━━━━━━ 44s 6ms/step - loss: 6.0932 - val_loss: 7.6081 - learning_rate: 0.0010
Epoch 2/20
5929/5929 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 270705672192.0000 - val_loss: 12063048832074645504.0000 - learning_rate: 0.0010
Epoch 3/20
5929/5929 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 1047178706944.0000 - val_loss: 7.8175 - learning_rate: 0.0010
Epoch 4/20
5929/5929 ━━━━━━━━━━━━━━━━━━━━ 38s 3ms/step - loss: 4.3504 - val_loss: 7.5722 - learning_rate: 1.0000e-04
Epoch 5/20
5929/5929 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - loss: 5.7707 - val_loss: 7.5392 - learning_rate: 1.0000e-04
Epoch 6/20
5929/5929 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 4.7744 - val_loss: 7.4719 - learning_rate: 1.0000e-04
Epoch 7/20
5929/5929 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 4.5733 - val_loss: 7.4374 - learning_rate: 1.0000e-04
Epoch 8/20
5929/5929 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 4.1967 - val_loss: 7.3939 - learning_rate: 1.0000e-04
Epoch 9/20
5929/5929 ━━━━━━━━━━━━━

In [7]:
# Cell 6: Compute Threshold from Train Normals
errs_train = np.mean((vae.predict(X_train) - X_train)**2, axis=1)
thr = np.percentile(errs_train[normal_mask], 95)
print("Threshold (95th percentile):", thr)


7500/7500 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step
Threshold (95th percentile): 0.38612333


In [8]:
# Cell 7: Evaluate on Test Set (VAE + Zero-Shot)
# VAE-only flags
errs_test = np.mean((vae.predict(X_test) - X_test)**2, axis=1)
flags_test = (errs_test > thr).astype(int)

# Zero-shot classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0 if torch.cuda.is_available() else -1)
candidate_labels = ["normal","anomaly"]

# Zero-shot on flagged only
zs_flags = np.zeros_like(flags_test)
for idx in np.where(flags_test==1)[0]:
    text = f"Time {int(test_df.iloc[idx]['ts'])}: PID {int(test_df.iloc[idx]['PID'])}, " + \
           f"{int(test_df.iloc[idx]['MINFLT'])} minor faults, {int(test_df.iloc[idx]['MAJFLT'])} major faults, " + \
           f"{test_df.iloc[idx]['MEM']*100:.1f}% memory"
    res = classifier(text, candidate_labels, hypothesis_template="This record is {}.", multi_label=False)
    zs_flags[idx] = 1 if res["labels"][0]=="anomaly" else 0

# Final test predictions
final_test = flags_test & zs_flags

print("Test VAE-only:", accuracy_score(y_test, flags_test))
print("Test Pipeline :", accuracy_score(y_test, final_test))
print("Test Confusion:\n", confusion_matrix(y_test, final_test))
print("Test Report:\n", classification_report(y_test, final_test))


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Test VAE-only: 0.8421
Test Pipeline : 0.8628
Test Confusion:
 [[51576  1126]
 [ 7106   192]]
Test Report:
               precision    recall  f1-score   support

           0       0.88      0.98      0.93     52702
           1       0.15      0.03      0.04      7298

    accuracy                           0.86     60000
   macro avg       0.51      0.50      0.49     60000
weighted avg       0.79      0.86      0.82     60000



In [9]:
# Cell 8: Final Validation on Separate CSV
val_df = pd.read_csv("sampled_data1.csv", low_memory=False)
val_df = val_df.dropna(subset=features+['type']).reset_index(drop=True)

X_val = scaler.transform(val_df[features].astype(float)).astype(np.float32)
y_val = val_df['type'].astype(int).values

# VAE-only on validation
errs_val = np.mean((vae.predict(X_val)-X_val)**2, axis=1)
flags_val = (errs_val > thr).astype(int)

print("Validation VAE-only:", accuracy_score(y_val, flags_val))
print("Validation Confusion:\n", confusion_matrix(y_val, flags_val))
print("Validation Report:\n", classification_report(y_val, flags_val))

# Zero-shot on every validation record
zs_val = np.zeros_like(flags_val)
for idx in range(len(val_df)):
    text = val_df.apply(lambda r: f"Time {int(r.ts)}: PID {int(r.PID)}, {int(r.MINFLT)} minor faults, {int(r.MAJFLT)} major faults, {r.MEM*100:.1f}% memory", axis=1).iloc[idx]
    res = classifier(text, candidate_labels, hypothesis_template="This record is {}.", multi_label=False)
    zs_val[idx] = 1 if res["labels"][0]=="anomaly" else 0

print("Validation Zero-Shot:", accuracy_score(y_val, zs_val))
print("Validation Confusion:\n", confusion_matrix(y_val, zs_val))
print("Validation Report:\n", classification_report(y_val, zs_val))


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Validation VAE-only: 0.829
Validation Confusion:
 [[823  45]
 [126   6]]
Validation Report:
               precision    recall  f1-score   support

           0       0.87      0.95      0.91       868
           1       0.12      0.05      0.07       132

    accuracy                           0.83      1000
   macro avg       0.49      0.50      0.49      1000
weighted avg       0.77      0.83      0.79      1000

Validation Zero-Shot: 0.47
Validation Confusion:
 [[400 468]
 [ 62  70]]
Validation Report:
               precision    recall  f1-score   support

           0       0.87      0.46      0.60       868
           1       0.13      0.53      0.21       132

    accuracy                           0.47      1000
   macro avg       0.50      0.50      0.41      1000
weighted avg       0.77      0.47      0.55      1000

